In [1]:
#Import Libraries
import os
import calendar
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

#Load data
df_prd_div = pd.read_excel('gs://mso-hw/RT_CVR_PRD&DIV.xlsx')

# Exploratory Data Analysis

In [2]:
#View few rows
df_prd_div.head(3)

,Acctg_dt,Coverage_code,Channel,Division,product,Creates,Cancels,Completes,Response_time,Conversion_Rate
0,2019-02-01,D2C,Online,46,REFRIGERATION,139,50.0,88,4.208633,0.633094
1,2019-02-01,HW Init,Online,22,DISH,1,NaN,1,1.000000,1.000000
2,2019-02-01,D2C,Online,9,GDO,5,2.0,3,1.400000,0.600000


In [3]:
#Check data type for each column
df_prd_div.dtypes

Acctg_dt           datetime64[ns]
Coverage_code              object
Channel                    object
Division                    int64
product                    object
Creates                     int64
Cancels                   float64
Completes                   int64
Response_time             float64
Conversion_Rate           float64
dtype: object

In [4]:
#Check for missing values
df_prd_div.isnull().sum()

Acctg_dt              0
Coverage_code         0
Channel               0
Division              0
product               0
Creates               0
Cancels            3177
Completes             0
Response_time         0
Conversion_Rate       0
dtype: int64

In [5]:
#Calculate percentage missing values in each column
round(100*(df_prd_div.isnull().sum()/len(df_prd_div.index)), 2)

Acctg_dt            0.00
Coverage_code       0.00
Channel             0.00
Division            0.00
product             0.00
Creates             0.00
Cancels            21.12
Completes           0.00
Response_time       0.00
Conversion_Rate     0.00
dtype: float64

In [6]:
#Calculate row counts by Division and Product Types
prd_and_div = df_prd_div.pivot_table(index=['Division','product'], aggfunc='size')
print(prd_and_div)

Division  product      
6         RECREATION        774
9         GDO               747
          TOOLS             106
10        COOL                1
          HVAC              921
          OIL-BOILER         99
20        COOKING MICRO     134
22        COOKING          1209
          COOKING MICRO     713
          DISH             1191
          DISPOSER          567
26        DRYER            1278
          WASHER           1344
32        COOL                1
          PORTHEATER          6
42        COOL              408
          WATERHEAT         772
          WATERTREAT        720
46        REFRIGERATION    1373
          Unknown            55
57        AV                  4
          TV                 77
          TV DIGITAL        419
71        GRILLING          450
          LAWN & GARDEN     732
          TRACTOR           934
83        WATERTREAT          5
dtype: int64


# Analysis - Statistical Inference

In [7]:
#Drop Cancels column
df = df_prd_div.copy()
df.drop('Cancels', axis=1, inplace=True)

#Split mso and online into d2c and hw
df = df[df["Division"].isin([22,26,46])] #Filter for three Divisions:- DIV22(Dishwasher), 
                                         #DIV26(Dryer, Washer), and DIV46(Refrigeration)
d2c_mso = (df[df["Coverage_code"].isin(["D2C"]) & df["Channel"].isin(["MSO"])])
hw_mso = (df[df["Coverage_code"].isin(["HW Init"]) & df["Channel"].isin(["MSO"])])
d2c_online = (df[df["Coverage_code"].isin(["D2C"]) & df["Channel"].isin(["Online"])])
hw_online = (df[df["Coverage_code"].isin(["HW Init"]) & df["Channel"].isin(["Online"])])

#Calculate the min, mean and max values for response time (RT)
rt_mean_min_max = df.groupby(['Coverage_code', 'Channel', 'Division']).agg({'Response_time': 
                                                                            ['mean', 'min', 'max']})
rt_mean_min_max.columns = ['RT_Mean', 'RT_Min', 'RT_Max']
rt_mean_min_max = rt_mean_min_max.reset_index()

#Calculate the min, mean and max values for conversion rate (CVR)
cvr_mean_min_max = df.groupby(['Coverage_code', 'Channel', 'Division']).agg({'Conversion_Rate': 
                                                                             ['mean', 'min', 'max']})
cvr_mean_min_max.columns = ['CVR_Mean', 'CVR_Min', 'CVR_Max']
cvr_mean_min_max = cvr_mean_min_max.reset_index()

In [8]:
#Print Response Time result 
print(round(rt_mean_min_max),1)

   Coverage_code Channel  Division  RT_Mean  RT_Min  RT_Max
0            D2C     MSO        22      4.0     0.0    25.0
1            D2C     MSO        26      3.0     1.0     9.0
2            D2C     MSO        46      5.0     0.0    19.0
3            D2C  Online        22      3.0     0.0    12.0
4            D2C  Online        26      3.0     0.0    13.0
5            D2C  Online        46      4.0     2.0    13.0
6        HW Init     MSO        22      3.0     0.0    17.0
7        HW Init     MSO        26      3.0     0.0    31.0
8        HW Init     MSO        46      5.0     1.0    24.0
9        HW Init  Online        22      3.0     0.0    12.0
10       HW Init  Online        26      2.0     0.0    17.0
11       HW Init  Online        46      4.0     0.0    21.0 1


In [10]:
#Print Conversion Rate result
print(cvr_mean_min_max)

   Coverage_code Channel  Division  CVR_Mean   CVR_Min  CVR_Max
0            D2C     MSO        22  0.557899  0.045455      1.0
1            D2C     MSO        26  0.650636  0.433225      0.9
2            D2C     MSO        46  0.671580  0.434579      1.0
3            D2C  Online        22  0.705575  0.250000      1.0
4            D2C  Online        26  0.697792  0.400000      1.0
5            D2C  Online        46  0.638599  0.250000      1.0
6        HW Init     MSO        22  0.865862  0.250000      1.0
7        HW Init     MSO        26  0.866897  0.250000      1.0
8        HW Init     MSO        46  0.831706  0.333333      1.0
9        HW Init  Online        22  0.989418  0.500000      1.0
10       HW Init  Online        26  0.953235  0.333333      1.0
11       HW Init  Online        46  0.924837  0.333333      1.0


## Correlation Analysis

In [11]:
#DIV22 (Dishwasher)
#MSO_D2C
d2c_mso_div22 = d2c_mso[d2c_mso["Division"].isin([22])]
d2c_mso_div22_cor = d2c_mso_div22[['Conversion_Rate', 'Response_time']].corr()

#MSO_HW
hw_mso_div22 = hw_mso[hw_mso["Division"].isin([22])]
hw_mso_div22_cor = hw_mso_div22[['Conversion_Rate', 'Response_time']].corr()

#Online_D2C
d2c_online_div22 = d2c_online[d2c_online["Division"].isin([22])]
d2c_online_div22_cor = d2c_online_div22[['Conversion_Rate', 'Response_time']].corr()

#Online_HW
hw_online_div22 = hw_online[hw_online["Division"].isin([22])]
hw_online_div22_cor = hw_online_div22[['Conversion_Rate', 'Response_time']].corr()

#DIV26 (Dryer and Washer)
#MSO_D2C 
d2c_mso_div26 = d2c_mso[d2c_mso["Division"].isin([26])]
d2c_mso_div26_cor = d2c_mso_div26[['Conversion_Rate', 'Response_time']].corr()

#MSO_HW 
hw_mso_div26 = hw_mso[hw_mso["Division"].isin([26])]
hw_mso_div26_cor = hw_mso_div26[['Conversion_Rate', 'Response_time']].corr()

#Online_D2C  
d2c_online_div26 = d2c_online[d2c_online["Division"].isin([26])]
d2c_online_div26_cor = d2c_online_div26[['Conversion_Rate', 'Response_time']].corr()

#Online_HW 
hw_online_div26 = hw_online[hw_online["Division"].isin([26])]
hw_online_div26_cor = hw_online_div26[['Conversion_Rate', 'Response_time']].corr()

#DIV46 (Refrigeration)
#MSO_D2C 
d2c_mso_div46 = d2c_mso[d2c_mso["Division"].isin([46])]
d2c_mso_div46_cor = d2c_mso_div46[['Conversion_Rate', 'Response_time']].corr()

#MSO_HW
hw_mso_div46 = hw_mso[hw_mso["Division"].isin([46])]
hw_mso_div46_cor = hw_mso_div46[['Conversion_Rate', 'Response_time']].corr()

#Online 
d2c_online_div46 = d2c_online[d2c_online["Division"].isin([46])]
d2c_online_div46_cor = d2c_online_div46[['Conversion_Rate', 'Response_time']].corr()

hw_online_div46 = hw_online[hw_online["Division"].isin([46])]
hw_online_div46_cor = hw_online_div46[['Conversion_Rate', 'Response_time']].corr()

In [13]:
#Show results for MSO Division 22 (Dish, Cooking, Cooking Micro,Disposer)
print(d2c_mso_div22_cor), print(hw_mso_div22_cor)

                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.060162
Response_time          -0.060162       1.000000
                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.199078
Response_time          -0.199078       1.000000


(None, None)

In [14]:
#Show results for Online Division 22 (Dish, Cooking, Cooking Micro,Disposer)
print(d2c_online_div22_cor), print(hw_online_div22_cor)

                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.006583
Response_time          -0.006583       1.000000
                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.068612
Response_time          -0.068612       1.000000


(None, None)

In [15]:
#Show results for MSO Division 26  (Dryer and Washer)
print(d2c_mso_div26_cor), print(hw_mso_div26_cor)

                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.303218
Response_time          -0.303218       1.000000
                 Conversion_Rate  Response_time
Conversion_Rate         1.000000       0.000159
Response_time           0.000159       1.000000


(None, None)

In [16]:
#Show results for Online Division 26  (Dryer and Washer)
print(d2c_online_div26_cor), print(hw_online_div26_cor)

                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.280974
Response_time          -0.280974       1.000000
                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.068124
Response_time          -0.068124       1.000000


(None, None)

In [17]:
#Show results for MSO Division 46 (Refrigeration)
print(d2c_mso_div46_cor), print(hw_mso_div46_cor)

                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.410749
Response_time          -0.410749       1.000000
                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.113472
Response_time          -0.113472       1.000000


(None, None)

In [19]:
#Show results for Online Division 46 (Refrigeration)
print(d2c_online_div46_cor), print(hw_online_div46_cor)

                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.475605
Response_time          -0.475605       1.000000
                 Conversion_Rate  Response_time
Conversion_Rate         1.000000      -0.047062
Response_time          -0.047062       1.000000


(None, None)

# Build Model

To determine the degree (strength) of relationship between KPIs (RT vs CVR)

In [22]:
# Linear Regression Model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score,r2_score

#DIV22 (Dish, Cooking, Cooking Micro,Disposer)
X = d2c_mso_div22['Response_time'].values.reshape(-1,1)
y = d2c_mso_div22['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_Div22_MSO_D2C = {:.5} + {:.5}RT_Div22_MSO_D2C".format(reg.intercept_[0], 
                                                                                      reg.coef_[0][0]))

The linear model is: CVR_Div22_MSO_D2C = 0.58439 + -0.0072224RT_Div22_MSO_D2C


In [23]:
#DIV22 (Dish, Cooking, Cooking Micro,Disposer)
X = hw_mso_div22['Response_time'].values.reshape(-1,1)
y = hw_mso_div22['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_Div22_MSO_HW = {:.5} + {:.5}RT_Div22_MSO_HW".format(reg.intercept_[0], 
                                                                                      reg.coef_[0][0]))

The linear model is: CVR_Div22_MSO_HW = 0.91776 + -0.017208RT_Div22_MSO_HW


In [24]:
#DIV22 (Dish, Cooking, Cooking Micro,Disposer)
X = d2c_online_div22['Response_time'].values.reshape(-1,1)
y = d2c_online_div22['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_Div22_Online_D2C = {:.5} + {:.5}RT_Div22_Online_D2C".format(reg.intercept_[0], 
                                                                                            reg.coef_[0][0]))

The linear model is: CVR_Div22_Online_D2C = 0.70758 + -0.00070994RT_Div22_Online_D2C


In [25]:
#DIV22 (Dish, Cooking, Cooking Micro,Disposer)
X = hw_online_div22['Response_time'].values.reshape(-1,1)
y = hw_online_div22['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_Div22_Online_HW = {:.5} + {:.5}RT_Div22_Online_HW".format(reg.intercept_[0], 
                                                                                          reg.coef_[0][0]))

The linear model is: CVR_Div22_Online_HW = 0.99439 + -0.0017706RT_Div22_Online_HW


In [26]:
#DIV26 (Dryer and Washer)
X = d2c_mso_div26['Response_time'].values.reshape(-1,1)
y = d2c_mso_div26['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV26__MSO_D2C = {:.5} + {:.5}RT_DIV26__MSO_D2C".format(reg.intercept_[0], 
                                                                                        reg.coef_[0][0]))

The linear model is: CVR_DIV26__MSO_D2C = 0.71802 + -0.02054RT_DIV26__MSO_D2C


In [27]:
#DIV26 (Dryer and Washer)
X = hw_mso_div26['Response_time'].values.reshape(-1,1)
y = hw_mso_div26['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV26_MSO_HW = {:.5} + {:.5}RT_DIV26_MSO_HW".format(reg.intercept_[0], 
                                                                                      reg.coef_[0][0]))

The linear model is: CVR_DIV26_MSO_HW = 0.86686 + 1.1314e-05RT_DIV26_MSO_HW


In [28]:
#DIV26 (Dryer and Washer)
X = d2c_online_div26['Response_time'].values.reshape(-1,1)
y = d2c_online_div26['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV26_D2C_Online = {:.5} + {:.5}RT_DIV26_D2C_Online".format(reg.intercept_[0], 
                                                                                            reg.coef_[0][0]))

The linear model is: CVR_DIV26_D2C_Online = 0.74748 + -0.019021RT_DIV26_D2C_Online


In [29]:
#DIV26 (Dryer and Washer)
X = hw_online_div26['Response_time'].values.reshape(-1,1)
y = hw_online_div26['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV26_HW_Online = {:.5} + {:.5}RT_DIV26_HW_Online".format(reg.intercept_[0], 
                                                                                          reg.coef_[0][0]))

The linear model is: CVR_DIV26_HW_Online = 0.96241 + -0.0037103RT_DIV26_HW_Online


In [30]:
#DIV46 (Refrigeration)
X = d2c_mso_div46['Response_time'].values.reshape(-1,1)
y = d2c_mso_div46['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV46_D2C_MSO = {:.5} + {:.5}RT_DIV46_D2C_MSO".format(reg.intercept_[0], 
                                                                                      reg.coef_[0][0]))

The linear model is: CVR_DIV46_D2C_MSO = 0.79473 + -0.025026RT_DIV46_D2C_MSO


In [31]:
#DIV46 (Refrigeration)
X = hw_mso_div46['Response_time'].values.reshape(-1,1)
y = hw_mso_div46['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV46_HW_MSO = {:.5} + {:.5}RT_DIV46_HW_MSO".format(reg.intercept_[0], 
                                                                                    reg.coef_[0][0]))

The linear model is: CVR_DIV46_HW_MSO = 0.85549 + -0.005046RT_DIV46_HW_MSO


In [32]:
#DIV46 (Refrigeration)
X = d2c_online_div46['Response_time'].values.reshape(-1,1)
y = d2c_online_div46['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV46_D2C_Online = {:.5} + {:.5}RT_DIV46_D2C_Online".format(reg.intercept_[0], 
                                                                                            reg.coef_[0][0]))

The linear model is: CVR_DIV46_D2C_Online = 0.73168 + -0.023131RT_DIV46_D2C_Online


In [33]:
#DIV46 (Refrigeration)
X = hw_online_div46['Response_time'].values.reshape(-1,1)
y = hw_online_div46['Conversion_Rate'].values.reshape(-1,1)
reg = LinearRegression()
reg.fit(X, y)
print("The linear model is: CVR_DIV46_HW_Online = {:.5} + {:.5}RT_DIV46_HW_Online".format(reg.intercept_[0], 
                                                                                      reg.coef_[0][0]))

The linear model is: CVR_DIV46_HW_Online = 0.93376 + -0.0021802RT_DIV46_HW_Online


# Interpretation of Results:

The sign of a regression coefficient tells you whether there is a positive or negative correlation between each independent variable and the dependent variable. A negative coefficient suggests that as the independent variable (RT) increases, the dependent variable (CVR) tends to decrease. 

Overall, Response Time (RT) shows a negative coefficient across all divisions which implies that wnen the response time is low, we get a better conversion rate. The degree of conversion rate depend on the channel, coverage code and division (product types).
